In [16]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Date, Numeric, DateTime
from sqlalchemy.orm import scoped_session, sessionmaker
from sqlalchemy.ext.automap  import automap_base

In [17]:
# The code was removed by Watson Studio for sharing.

In [18]:
#DB connection string
connection_string = "db2+ibm_db://" + user + ":" + pwd + "@dashdb-txn-flex-yp-dal13-43.services.dal.bluemix.net:50000/BLUDB"

In [19]:

engine = create_engine(connection_string, echo=False)
meta = MetaData()

nome_tabela = 'COVID-BR'

covid = Table('COVID-BR', meta, 
    Column("COUNTRY", String(30)),
    Column("PROVINCE_STATE", String(50)),
    Column("CITY", String(50)),
    Column("IBGEID", Integer),
    Column("COD_REGIAODESAUDE",Numeric(18,8)),
    Column("NOME_REGIAODESAUDE",String(50)),
    Column("DEATHS", Integer),
    Column("CASES", Integer),
    Column("DEATHS_PER_100K_INHABITANTS", Numeric(18,8)),
    Column("TOTALCASES_PER_100K_INHABITANTS", Numeric(18,8)),
    Column("DEATHS_BY_TOTALCASES", Numeric(18,8)),
    Column("SOURCE", String(10)),
    Column("DATE", Date),
    Column("NEWCASES", Integer),
    Column("NEWDEATHS",Integer))

# date	newCases	newDeaths
#'DEATHS_PER_100K_INHABITANTS','TOTALCASES_PER_100K_INHABITANTS','DEATHS_BY_TOTALCASES'

dropTable = True

if not engine.dialect.has_table(engine, nome_tabela, schema = 'BLUADMIN'):
    meta.create_all(engine)
else:
    if dropTable:
        covid.drop(engine) # Apaga tabela
        meta.create_all(engine) # Cria tabela
    conn = engine.connect()
    #stmt = covid.delete().where(students.c.lastname == 'Khanna')
    stmt = covid.delete()
    conn.execute(stmt)
    conn.close()

In [20]:
Base = automap_base()

#engine = create_engine(connection_string, echo=True)
session = scoped_session(sessionmaker(autocommit=True,
                                      autoflush=False,
                                      bind=engine))

Base.prepare(engine, reflect=True)
base_sa = Base.classes

In [21]:
df = pd.read_csv('https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-cities.csv')
df.head()

,country,state,city,ibgeID,cod_RegiaoDeSaude,name_RegiaoDeSaude,deaths,totalCases,deaths_per_100k_inhabitants,totalCases_per_100k_inhabitants,deaths_by_totalCases,_source,date,newCases,newDeaths
0,Brazil,GO,Abadia de Goiás/GO,5200050,52001.0,Central,0,33,0.00000,376.15411,0.00000,SES,2020-06-26,1,0
1,Brazil,MG,Abadia dos Dourados/MG,3100104,31074.0,Patrocínio / Monte Carmelo,0,5,0.00000,71.54099,0.00000,SES,2020-06-26,0,0
2,Brazil,GO,Abadiânia/GO,5200100,52011.0,Pirineus,1,37,4.98952,184.61231,0.02703,SES,2020-06-26,2,1
3,Brazil,PA,Abaetetuba/PA,1500107,15011.0,Tocantins,97,2276,61.50997,1443.26497,0.04262,SES,2020-06-26,5,0
4,Brazil,MG,Abaeté/MG,3100203,31024.0,Sete Lagoas,0,8,0.00000,34.42785,0.00000,SES,2020-06-26,6,0


In [22]:
df.columns = ['COUNTRY','PROVINCE_STATE','CITY',
                     'IBGEID','COD_REGIAODESAUDE','NOME_REGIAODESAUDE','DEATHS','CASES','DEATHS_PER_100K_INHABITANTS',
              'TOTALCASES_PER_100K_INHABITANTS','DEATHS_BY_TOTALCASES','SOURCE','DATE',
              'NEWCASES','NEWDEATHS']
df.head()

,COUNTRY,PROVINCE_STATE,CITY,IBGEID,COD_REGIAODESAUDE,NOME_REGIAODESAUDE,DEATHS,CASES,DEATHS_PER_100K_INHABITANTS,TOTALCASES_PER_100K_INHABITANTS,DEATHS_BY_TOTALCASES,SOURCE,DATE,NEWCASES,NEWDEATHS
0,Brazil,GO,Abadia de Goiás/GO,5200050,52001.0,Central,0,33,0.00000,376.15411,0.00000,SES,2020-06-26,1,0
1,Brazil,MG,Abadia dos Dourados/MG,3100104,31074.0,Patrocínio / Monte Carmelo,0,5,0.00000,71.54099,0.00000,SES,2020-06-26,0,0
2,Brazil,GO,Abadiânia/GO,5200100,52011.0,Pirineus,1,37,4.98952,184.61231,0.02703,SES,2020-06-26,2,1
3,Brazil,PA,Abaetetuba/PA,1500107,15011.0,Tocantins,97,2276,61.50997,1443.26497,0.04262,SES,2020-06-26,5,0
4,Brazil,MG,Abaeté/MG,3100203,31024.0,Sete Lagoas,0,8,0.00000,34.42785,0.00000,SES,2020-06-26,6,0


In [23]:
nome_tabela = 'COVID-BR'

In [24]:
df.to_sql(name=nome_tabela, con=engine, if_exists='append', index=False)
#df.to_sql(name=nome_tabela, con=engine, if_exists='append', index=False, chunksize=1000)   # With number of records to be processed each time

In [25]:
df2 = pd.read_sql_table(nome_tabela, engine)
df2.head()

,country,province_state,city,ibgeid,cod_regiaodesaude,nome_regiaodesaude,deaths,cases,deaths_per_100k_inhabitants,totalcases_per_100k_inhabitants,deaths_by_totalcases,SOURCE,DATE,newcases,newdeaths
0,Brazil,GO,Abadia de Goiás/GO,5200050,52001.0,Central,0,33,0.00000,376.15411,0.00000,SES,2020-06-26,1,0
1,Brazil,MG,Abadia dos Dourados/MG,3100104,31074.0,Patrocínio / Monte Carmelo,0,5,0.00000,71.54099,0.00000,SES,2020-06-26,0,0
2,Brazil,GO,Abadiânia/GO,5200100,52011.0,Pirineus,1,37,4.98952,184.61231,0.02703,SES,2020-06-26,2,1
3,Brazil,PA,Abaetetuba/PA,1500107,15011.0,Tocantins,97,2276,61.50997,1443.26497,0.04262,SES,2020-06-26,5,0
4,Brazil,MG,Abaeté/MG,3100203,31024.0,Sete Lagoas,0,8,0.00000,34.42785,0.00000,SES,2020-06-26,6,0


In [26]:
session.close()